In [1]:
%pip install groq
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from groq import Groq
from dotenv import load_dotenv
import re
import os
from difflib import unified_diff
load_dotenv()
from langchain_groq import ChatGroq

client = Groq(api_key=os.getenv("GROQ_API_KEY"))


llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=4096,
)

In [3]:
inventory_entity = """
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/model/InventoryEntity.java"
Source file contents:
```java
package com.redhat.coolstore.model;

import java.io.Serializable;

import javax.persistence.Column;
import javax.persistence.Entity;
import javax.persistence.Id;
import javax.persistence.Table;
import javax.persistence.UniqueConstraint;
import javax.xml.bind.annotation.XmlRootElement;

@Entity
@XmlRootElement
@Table(name = "INVENTORY", uniqueConstraints = @UniqueConstraint(columnNames = "itemId"))
public class InventoryEntity implements Serializable {

	private static final long serialVersionUID = 7526472295622776147L; 

    @Id
    private String itemId;


    @Column
    private String location;


    @Column
    private int quantity;


    @Column
    private String link;

    public InventoryEntity() {

    }

    public String getItemId() {
		return itemId;
	}

	public void setItemId(String itemId) {
		this.itemId = itemId;
	}

	public String getLocation() {
		return location;
	}

	public void setLocation(String location) {
		this.location = location;
	}

	public int getQuantity() {
		return quantity;
	}

	public void setQuantity(int quantity) {
		this.quantity = quantity;
	}

	public String getLink() {
		return link;
	}

	public void setLink(String link) {
		this.link = link;
	}

	@Override
    public String toString() {
        return "InventoryEntity [itemId=" + itemId + ", availability=" + quantity + "/" + location + " link=" + link + "]";
    }
}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 5
### incident 1
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 6
### incident 2
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 7
### incident 3
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 8
### incident 4
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 9
### incident 5
incident to fix: "Replace the `javax.xml` import statement with `jakarta.xml`"
Line number: 10

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.
"""

In [4]:
shopping_cart_services = """
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/service/ShoppingCartService.java"
Source file contents:
```java
package com.redhat.coolstore.service;

import java.util.Hashtable;
import java.util.logging.Logger;

import javax.ejb.Stateful;
import javax.inject.Inject;
import javax.naming.Context;
import javax.naming.InitialContext;
import javax.naming.NamingException;

import com.redhat.coolstore.model.Product;
import com.redhat.coolstore.model.ShoppingCart;
import com.redhat.coolstore.model.ShoppingCartItem;

@Stateful
public class ShoppingCartService  {

    @Inject
    Logger log;

    @Inject
    ProductService productServices;

    @Inject
    PromoService ps;


    @Inject
    ShoppingCartOrderProcessor shoppingCartOrderProcessor;

    private ShoppingCart cart  = new ShoppingCart(); //Each user can have multiple shopping carts (tabbed browsing)

   

    public ShoppingCartService() {
    }

    public ShoppingCart getShoppingCart(String cartId) {
        return cart;
    }

    public ShoppingCart checkOutShoppingCart(String cartId) {
        ShoppingCart cart = this.getShoppingCart(cartId);
      
        log.info("Sending  order: ");
        shoppingCartOrderProcessor.process(cart);
   
        cart.resetShoppingCartItemList();
        priceShoppingCart(cart);
        return cart;
    }

    public void priceShoppingCart(ShoppingCart sc) {

        if (sc != null) {

            initShoppingCartForPricing(sc);

            if (sc.getShoppingCartItemList() != null && sc.getShoppingCartItemList().size() > 0) {

                ps.applyCartItemPromotions(sc);

                for (ShoppingCartItem sci : sc.getShoppingCartItemList()) {

                    sc.setCartItemPromoSavings(
                            sc.getCartItemPromoSavings() + sci.getPromoSavings() * sci.getQuantity());
                    sc.setCartItemTotal(sc.getCartItemTotal() + sci.getPrice() * sci.getQuantity());

                }

                sc.setShippingTotal(lookupShippingServiceRemote().calculateShipping(sc));

                if (sc.getCartItemTotal() >= 25) {
                    sc.setShippingTotal(sc.getShippingTotal()
                            + lookupShippingServiceRemote().calculateShippingInsurance(sc));
                }

            }

            ps.applyShippingPromotions(sc);

            sc.setCartTotal(sc.getCartItemTotal() + sc.getShippingTotal());

        }

    }

    private void initShoppingCartForPricing(ShoppingCart sc) {

        sc.setCartItemTotal(0);
        sc.setCartItemPromoSavings(0);
        sc.setShippingTotal(0);
        sc.setShippingPromoSavings(0);
        sc.setCartTotal(0);

        for (ShoppingCartItem sci : sc.getShoppingCartItemList()) {
            Product p = getProduct(sci.getProduct().getItemId());
            //if product exist
            if (p != null) {
                sci.setProduct(p);
                sci.setPrice(p.getPrice());
            }

            sci.setPromoSavings(0);
        }

    }

    public Product getProduct(String itemId) {
        return productServices.getProductByItemId(itemId);
    }

	private static ShippingServiceRemote lookupShippingServiceRemote() {
        try {
            final Hashtable<String, String> jndiProperties = new Hashtable<>();
            jndiProperties.put(Context.INITIAL_CONTEXT_FACTORY, "org.wildfly.naming.client.WildFlyInitialContextFactory");

            final Context context = new InitialContext(jndiProperties);

            return (ShippingServiceRemote) context.lookup("ejb:/ROOT/ShippingService!" + ShippingServiceRemote.class.getName());
        } catch (NamingException e) {
            throw new RuntimeException(e);
        }
    }
}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 6
Solved example diff:
```diff


### incident 1
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 7
Solved example diff:
```diff


### incident 2
incident to fix: "Stateful EJBs can be converted to a CDI bean by replacing the `@Stateful` annotation with a bean-defining annotation
 that encompasses the appropriate scope (e.g., `@ApplicationScoped`). `@Stateful` EJBs often translate to `@SessionScoped`
 beans (a scope which requires activating the `quarkus-undertow` extension), but the appropriate scope may differ based
 on your application architecture. Review your application's requirements to determine the appropriate scope.

 Note that it is recommended, as a good practice, to keep state external from the service in Quarkus."
Line number: 16
Solved example diff:
```diff



# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.
"""

In [5]:
order_item_='''
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/model/OrderItem.java"
Source file contents:
```java
package com.redhat.coolstore.model;

import java.io.Serializable;

import javax.persistence.Column;
import javax.persistence.Entity;
import javax.persistence.GeneratedValue;
import javax.persistence.Id;
import javax.persistence.Table;

@Entity
@Table(name = "ORDER_ITEMS")
public class OrderItem implements Serializable {
	private static final long serialVersionUID = 64565445665456666L;

	@Id
	@Column(name="ID")
	@GeneratedValue
	private long id;

	private int quantity;

	private String productId;

	public OrderItem() {}

	public String getProductId() {
		return productId;
	}

	public void setProductId(String productId) {
		this.productId = productId;
	}

	public int getQuantity() {
		return quantity;
	}

	public void setQuantity(int quantity) {
		this.quantity = quantity;
	}

	@Override
	public String toString() {
		return "OrderItem [productId=" + productId + ", quantity=" + quantity + "]";
	}

}
```

## Issues

### incident 0
incident to fix: "The way in which Hibernate determines implicit names for sequences and tables associated with identifier generation has changed in 6.0 which may affect migrating applications. 
 As of 6.0, Hibernate by default creates a sequence per entity hierarchy instead of a single sequence hibernate_sequence. 
 Due to this change, users that previously used `@GeneratedValue(strategy = GenerationStrategy.AUTO)` or simply `@GeneratedValue` (since `AUTO` is the default), need to ensure that the database now contains sequences for every entity, named `<entity name>_seq`. For an entity Person, a sequence person_seq is expected to exist. 
 It’s best to run hbm2ddl (e.g. by temporarily setting `hbm2ddl.auto=create`) to obtain a list of DDL statements for the sequences."
Line number: 18
### incident 1
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 5
### incident 2
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 6
### incident 3
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 7
### incident 4
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 8
### incident 5
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 9

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

'''

In [6]:
def separate_sections(document):
    section_titles = ["## Reasoning", "## Updated File", "## Additional Information"]
    # Find the start index of each section by looking for the section titles, filter out not found (-1) indices
    indices = {
        title: document.find(title)
        for title in section_titles
        if document.find(title) != -1
    }

    sorted_indices = dict(sorted(indices.items(), key=lambda item: item[1]))
    sorted_indices["end"] = len(document)
    titles_sorted = list(sorted_indices.keys()) + ["end"]

    # Extract the sections based on the indices found
    sections = {}
    for i, title in enumerate(
        titles_sorted[:-1]
    ):  # Skip the last item ('end') for iteration
        start_index = sorted_indices[title] + len(title)
        end_title = titles_sorted[i + 1]
        end_index = sorted_indices[end_title]
        sections[title] = document[start_index:end_index].strip()

    return sections


def parse_file_solution_content(content: str):
    code_block_pattern = r"```(?:\w+)?\s+(.+?)```"

    sections = separate_sections(content)
    reasoning = sections.get("## Reasoning", "")
    updated_file_content = sections.get("## Updated File", "")
    additional_info = sections.get("## Additional Information", "")

    comments_pattern = r"(Rest of the code remains unchanged)|(rest of the code remains the same)|(Other methods remain unchanged)|(Rest of the class remains unchanged)|(Rest of the methods remain unchanged)"
    comments_pattern_matches = re.findall(
        comments_pattern, updated_file_content, re.DOTALL
    )
    unnecessary_comments = False
    if comments_pattern_matches:
        # print("Unnecessary comments found in the Updated file!")
        return True

    code_block_matches = re.findall(code_block_pattern, updated_file_content, re.DOTALL)
    matching_blocks = []
    for block in code_block_matches:
        matching_blocks.append(block)

    if matching_blocks:
        # If multiple matches default to first
        updated_file = matching_blocks[0].strip()
        if len(matching_blocks) > 1:
            return f"Multiple matching codeblocks found, defaulting to first"
        else:
            return f"Found single matching codeblock"
    elif code_block_matches:
        # fallback to first discovered codeblock
        updated_file = code_block_matches[0]
        if len(code_block_matches) > 1:
            return f"Multiple codeblocks found, defaulting to first"
        else:
            return f"Found single codeblock"
    else:
        updated_file = ""
        return "No codeblocks detected in LLM response"

In [7]:
def invoking(human_input: str, messages: list):
    messages.append(("human", human_input))
    ai_msg = llm.invoke(messages)
    return ai_msg.content

In [8]:
def merging_response(llm_response: str, input_file: str) -> str:
    diff = unified_diff(llm_response.splitlines(), input_file.splitlines(), lineterm="")
    with open("difference.txt", "w") as f:
        f.write("\n".join(list(diff)))

    comment_index = 1
    occurances_of_triple_backticks = []
    commented_line = ""
    cleaned_commented_line = ""
    with open("difference.txt", "r") as f:
        lines = f.readlines()

        for line in lines:
            if " Rest of " in line:
                comment_index += lines.index(line)
                commented_line += line

            if "```" in line:
                occurances_of_triple_backticks.append(lines.index(line))

    end_block = occurances_of_triple_backticks[1]
    final_extract = ""
    second_final_extract = ""
    stats_pattern = r"@@ -\d.+ @@"

    for i in lines[comment_index : end_block - 2]:
        stats_pattern_matches = re.findall(stats_pattern, i, re.DOTALL)
        if ("+" in i) and not stats_pattern_matches:
            second_final_extract += i

    for i in second_final_extract:
        if "+" not in i:
            final_extract += i

    for i in commented_line:
        if "-" not in i:
            cleaned_commented_line += i

    new_llm_response = llm_response.replace(f"{cleaned_commented_line}", final_extract)
    return new_llm_response


In [9]:
def ensure_codeblocks(prompt: str) -> str:
    """This function ensures that LLM response will have codeblocks along with complete code."""
    messages = [
        (
            "system",
            "You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.",
        ),
        ("human", prompt),
    ]
    ai_msg = llm.invoke(messages)
    content = ai_msg.content
    
    if parse_file_solution_content(content) == "No codeblocks detected in LLM response":
        print(content)
        print(invoking("Please provide a complete response.", messages))

    else:
        if parse_file_solution_content(content) == True:

            print(content)
            # This was a previous waay of getting full response, by invoking the llm response again
            # print(
            #     invoking(
            #         'Please generate a complete response, without unnecessary comments like "// ... Rest of the methods remain unchanged" or "// ... Rest of the code remain unchanged".',
            #         messages,
            #     )
            # )
            print(merging_response(content,prompt))
            

        else:
            print(content)

In [10]:
ensure_codeblocks(shopping_cart_services)

## Reasoning

1. **incident 0**: The `javax.ejb` package is part of the Java EE specification, while Quarkus uses the Jakarta EE specification. Therefore, we need to replace the `javax.ejb` import statement with `jakarta.ejb`.

2. **incident 1**: Similar to incident 0, we need to replace the `javax.inject` import statement with `jakarta.inject` as Quarkus uses the Jakarta EE specification.

3. **incident 2**: Stateful EJBs are not natively supported in Quarkus. Instead, we can use CDI beans with a suitable scope. In this case, we can replace the `@Stateful` annotation with `@SessionScoped` as stateful EJBs often translate to session-scoped beans. However, it is recommended to keep state external from the service in Quarkus.

## Updated File

```java
package com.redhat.coolstore.service;

import java.util.Hashtable;
import java.util.logging.Logger;

import jakarta.ejb.Stateful; // Changed to jakarta.ejb
import jakarta.inject.Inject; // Changed to jakarta.inject
import jakarta.naming.Con

In [ ]:
ensure_codeblocks(inventory_entity)

Below is a control prompt, that provides a response that includes code blocks and no unnecessary comments in the response.

In [ ]:
ensure_codeblocks(order_item_)